<h1><center>Laboratory Electronic Notebook</center></h1>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }/*This file contains any manual css for this page that needs to override the global styles. This is only required when different pages style the same element differently. This is just a hack to deal with our current css styles and no new styling should be added in this file.*/  #ipython-main-app {     position: relative; } #jupyter-main-app {     position: relative; }   /*Hemendik aurrera nik idatzi det*/ body {    font-family: Ubuntu, sans-serif; }  .jupyter-widgets { 	margin: 5px !important; 	font-family: Ubuntu, sans-serif !important; }  .jupyter-button { 	top: 4px !important; 	line-height: 28px !important; }  .widget-text { 	font-size: 15px !important; }  .output_subarea { 	margin: auto; }  /*formularioen stylea*/ /* naranja: #fcba05 */ /* gris: #afafaf */  input[type=text] { 	padding: 3px !important; 	border: 0 0 0 0; 	border-bottom-color: #fcba05 !important; 	border-bottom-width: 2px !important; 	 	border-top-width: 1px !important; 	border-left-width: 1px !important; 	border-right-width: 1px !important; 	 	border-top-color: transparent !important; 	border-left-color: transparent !important; 	border-right-color: transparent !important; }  input[type=text]:hover{ 	     border-top-color: #afafaf !important; 	border-left-color: #afafaf !important; 	border-right-color: #afafaf !important; }  input[type=text]:focus{     border-top-color: #fcba05 !important; 	border-left-color: #fcba05 !important; 	border-right-color: #fcba05 !important; }</style>"))

In [2]:
display(HTML("<style>.jupyter-button{margin-top:5px} .widget-hbox{min-height:50px} .widget-checkbox .widget-label{width:0px} .widget-checkbox .widget-dropdown{width:0px}"))

In [3]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

from bqplot import pyplot as pyplot

import numpy as np

import pandas as pd

from IPython.display import display, FileLink, FileLinks
import ipywidgets as widgets

import qgrid

from traitlets import link

import sys,os
import csv

import io
import fileupload

import matplotlib as mpl
import matplotlib.colors as mcolors  
import matplotlib.pyplot as plt      

from matplotlib import rc            

In [4]:
def isNotNumber(s):
    try:
        float(s)
    except:
        return True
    else:
        return False

In [5]:
class dataBase(object):
    
    def __init__(self):
        dataBase.all = {}
        dataBase.index = 0
        
    def addData(self, df, name=''):
        
        if name == '':
            name = "Data " + str(self.getLastName())
            self.incrementIndex()
        elif name in self.all.keys():
            name = name + " " + str(self.getLastName())
            self.incrementIndex()

        df = checkDFheader(df)
        self.all[name] = df
        refreshDataDropdown()
        dataDropdown.value = name
    
    def updateData(self, df, name):
        self.all[name] = df
        
    def getData(self, name):
        if name not in self.all.keys():
            return pd.DataFrame({"x": float(np.linspace(0.00, 0.00,1)), "y": np.linspace(0.00, 0.00,1)})
        else:        
            return self.all.get(name)
    
    def removeData(self, name):
        if len(self.all) > 1:
            del self.all[name]
        
    def getLastName(self):
        return self.index
    
    def incrementIndex(self):
        self.index = self.index + 1
        
    def show(self):
        return self.all
    
    def getDatas(self):
        return self.all.keys()



In [6]:
#Checks if the dataframe has a non-numerical header
def checkDFheader(df):
    
    header = list(df.columns)
    noHeader = True
    
    for elem in header:
        if isNotNumber(elem):
            noHeader = False
            break
            
    if noHeader:
        df2 = pd.DataFrame()
        for i in range(len(header)):
            df2[str(i)] = np.append([float(header[i])], df[header[i]])
        df = df2    
        
    return df

In [7]:
class plotBase(object):
    
    def __init__(self):
        plotBase.all = {}
        plotBase.plots = {}
        plotBase.simulations = {}
        plotBase.plotIndex = 0
        plotBase.simIndex = 0
        
    def addPlot(self, figure, name=''):
        if name == '':
            name = "Plot " + str(plotBase.plotIndex)
        elif name in plotBase.plots.keys():
            name = name + " " + str(plotBase.plotIndex)


        sc_x = figure.marks[3].scales.get('x')
        sc_y = figure.marks[3].scales.get('y')

        scale_x = bqm.LinearScale(min = sc_x.min, max = sc_x.max)
        scale_y = bqm.LinearScale(min = sc_y.min, max = sc_y.max)
        
        ax = figure.axes[0]
        ay = figure.axes[1]
        
        axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style= {'font-size': '15px'},
                num_ticks = ax.num_ticks,
                grid_lines = ax.grid_lines,
                grid_color = '#8e8e8e', 
                label=ax.label,
                label_location='middle',
                label_offset='50px')
        
        axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = ay.num_ticks,
                grid_lines = ay.grid_lines,
                grid_color = '#8e8e8e', 
                label=ay.label,
                orientation='vertical',
                label_location='middle',
                label_offset='50px')
        
        scatter = bqm.Scatter(x = figure.marks[3].x, 
                   y = figure.marks[3].y, 
                   scales = {'x': scale_x, 'y': scale_y}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   colors = figure.marks[3].colors,
                   )
        
        distances = bqm.Lines(x = figure.marks[0].x,
                 y = figure.marks[0].y,
                 scales = {'x': scale_x, 'y': scale_y},
                 line_style = 'dashed',
                 visible = figure.marks[0].visible,
                 colors = figure.marks[0].colors,
                 )
        
        line = bqm.Lines(x = figure.marks[2].x, 
                y = figure.marks[2].y, 
                scales = {'x': scale_x, 'y': scale_y}, 
                visible = figure.marks[2].visible,
                colors = figure.marks[2].colors,
                )
        
        scatterEst = bqm.Scatter(x = figure.marks[1].x,
                             y = figure.marks[1].y,
                             scales = {'x': scale_x, 'y': scale_y},
                             enable_move = False,
                             visible = figure.marks[1].visible,
                             colors = figure.marks[1].colors,
                            )

        fig = bq.Figure(title=figure.title,
                marks=[distances, scatterEst, line, scatter],
                axes=([axis_x, axis_y]),
                animation_duration=500,
                layout = widgets.Layout(width='90%', align_self='center'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
               )
        
        plotBase.all[name] = fig
        plotBase.plots[name] = fig
        
        refreshPlotDropdown()
        plotDropdown.value = name
        plotBase.plotIndex = plotBase.plotIndex + 1
        
    def addSimulation(self, figure, name='', replace=False):
        
        if not replace:
            if name == '':
                name = "Simulation " + str(plotBase.simIndex)

            elif name in plotBase.all.keys():
                name = name + " " + str(plotBase.simIndex)

        sc_x = figure.marks[3].scales.get('x')
        sc_y = figure.marks[3].scales.get('y')

        scale_x = bqm.LinearScale(min = sc_x.min, max = sc_x.max)
        scale_y = bqm.LinearScale(min = sc_y.min, max = sc_y.max)
            
        ax = figure.axes[0]
        ay = figure.axes[1]    
            
        axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = ax.num_ticks,
                grid_lines = ax.grid_lines,
                grid_color = '#8e8e8e', 
                label=ax.label,
                label_location='middle',
                label_offset='50px')
        
        axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = ay.num_ticks,
                grid_lines = ay.grid_lines,
                grid_color = '#8e8e8e', 
                label=ay.label,
                orientation='vertical',
                label_location='middle',
                label_offset='50px')
        
        scatter = bqm.Scatter(x = figure.marks[3].x, 
                   y = figure.marks[3].y, 
                   scales = {'x': scale_x, 'y': scale_y}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   colors = figure.marks[3].colors,
                   )
        
        distances = bqm.Lines(x = figure.marks[0].x,
                 y = figure.marks[0].y,
                 scales = {'x': scale_x, 'y': scale_y},
                 line_style = 'dashed',
                 visible = figure.marks[0].visible,
                 colors = figure.marks[0].colors,
                 )
        
        line = bqm.Lines(x = figure.marks[2].x, 
                y = figure.marks[2].y, 
                scales = {'x': scale_x, 'y': scale_y}, 
                visible = figure.marks[2].visible,
                colors = figure.marks[2].colors,
                )
        
        scatterEst = bqm.Scatter(x = figure.marks[1].x,
                             y = figure.marks[1].y,
                             scales = {'x': scale_x, 'y': scale_y},
                             enable_move = False,
                             visible = figure.marks[1].visible,
                             colors = figure.marks[1].colors,
                            )

        fig = bq.Figure(title=figure.title,
                marks=[distances, scatterEst, line, scatter],
                axes=([axis_x, axis_y]),
                animation_duration=500,
                layout = widgets.Layout(width='90%', align_self='center'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
               )
            
        plotBase.all[name] = fig
        plotBase.simulations[name] = fig
        
        refreshSimDropdown()

        simDropdown.value = name
        plotBase.simIndex = plotBase.simIndex + 1
        
    def getPlot(self, name):
        return plotBase.all.get(name)
    
    def removePlot(self, name):
        if len(self.plots) > 1:
            del self.all[name]
            del  self.plots[name]

    def removeSimulation(self, name):
        if len(self.simulations) > 1:
            del self.all[name]
            del  self.simulations[name]        
            
    def show(self):
        return plotBase.all
    
    def getPlots(self):
        return [elem for elem in plotBase.plots.keys()]
    
    def getSimulations(self):
        return [elem for elem in plotBase.simulations.keys()]

In [8]:
db = dataBase()

In [9]:
pb = plotBase()

In [10]:
fb = { '-Select file-': ''}

In [11]:
mainBlock = widgets.VBox([])

In [12]:
topBlock = widgets.VBox([], layout = widgets.Layout(border='solid 1px', padding='20px'))

In [13]:
header = widgets.HTML(
    value = "<h1>Start working: open your data</h1> <hr>"    
)

topBlock.children = [header,]

In [14]:
style_1 = {
    'description_width': 'initial',
          }

In [15]:
spacerText = widgets.HTML(
    value = "<br>"    
)

In [16]:
def uploadFile():
    """Create an upload files button that prints the file name and file size.
    """

    _upload_widget = fileupload.FileUploadWidget()

    def _cb(change):
        decoded = io.StringIO(change['owner'].data.decode('utf-8'))
        filename = change['owner'].filename

    _upload_widget.observe(_cb, names='data')
    return _upload_widget

uploadButton = uploadFile()

In [17]:
uploadBlock = widgets.VBox([])

uploadText = widgets.HTML(
    value = "<h2>Open your CSV file from your computer:</h2> <hr>"    
)

uploadDataName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

uploadedText = widgets.HTML(
    value = "",
)

uploadBlock.children = [uploadText, widgets.HBox([uploadDataName,uploadButton]), uploadedText]

In [18]:
topBlock.children = topBlock.children+(uploadBlock,)

In [19]:
def uploadCSV(change):
    
    try:
        uploadedText.value = "<b>Uploading file...</b>"
        datastr = str(uploadButton.data,'utf-8')
        datatbl = io.StringIO(datastr)
        dfNew = pd.read_csv(datatbl)
        db.addData(dfNew, uploadDataName.value)
        uploadedText.value = "<span style='color:green'><b>'"+ uploadButton.filename + "' file successfully loaded</b></span>"
        header.layout = {'display' : 'none'}
        refreshDataDropdown()
        if len(dataDropdown.options) > 0:
            bodyBlock.children = [chartBlock, plotBlock]
        mainBlock.children = [topBlock, bodyBlock, bottomBlock]
        liveUpdateCheckbox.value = False
        uploadDataName.value = ''    
        
    except Exception as e:
        uploadedText.value = "<span style='color:red'><b>An error has occurred, choose another file</b></span>"

In [20]:
uploadButton.observe(uploadCSV,'data')

In [21]:
csvBlock = widgets.VBox([])

csvBlock1 = widgets.HBox([])
csvBlock2 = widgets.HBox([])

csvText = widgets.HTML(
    value = "<h2>Open your CSV file from the server:</h2> <hr>"    
)

dataName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)



csvBlock1.children = [
    dataName]

filesDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose your file: </h4>',
    style = style_1
)


filePath = widgets.Text(
    value= '',
    placeholder="Type your file's path:",
    description="<h4>Path: </h4>",
    disabled=False,
    layout = {'width' : '80%'}
)

loadButton = widgets.ToggleButton(
    description='Open file',
    disabled=False,
    button_style='', 
    style = {
        'display': 'block',
        'margin': '0px auto',
        'height' : '28px',
        'border-radius' : '25px'
            }
)

loadedText = widgets.HTML(
    value = "",
)

csvBlock2.children = [
    filesDropdown,
    filePath,
    loadButton,
]

csvBlock.children = [
    csvText,
    csvBlock1,
    csvBlock2,
    loadedText
]

In [22]:
topBlock.children = topBlock.children+(csvBlock,)

In [23]:
def refreshDataDropdown():
    dataDropdown.options = db.getDatas()

In [24]:
def refreshFilesDropdown():
        
    root = os.getcwd()  
    for path, subdirs, files in os.walk(root):
        for name in files:
            if name[-4:] == '.csv':
                p = os.path.join(path, name)
                f = p.replace(root + os.sep, '')
                fb[f] = p
                
    filesDropdown.options = sorted(fb.keys())

In [25]:
refreshFilesDropdown()

In [26]:
def refreshPath(change):
    filePath.value = fb.get(filesDropdown.value)

In [27]:
filesDropdown.observe(refreshPath, 'value')

In [28]:
def openCSVFile(a):
    if loadButton.value == True:
        loadButton.value = False
        path = filePath.value
        if path == '':
            return None
        
        p = path.split()
        path = ''.join(p)
        
        try:
    
            data0 = pd.read_csv(str(path))       
            db.addData(data0, dataName.value)
            filePath.value = ''
            loadedText.value = "<span style='color:green'><b>File successfully loaded</b></span>"
            dataName.value = ''
            header.layout = {'display' : 'none'}
            refreshDataDropdown()
            if len(dataDropdown.options) > 0:
                bodyBlock.children = [chartBlock, plotBlock]
            mainBlock.children = [topBlock, bodyBlock, bottomBlock]
            liveUpdateCheckbox.value = False
            filesDropdown.value = filesDropdown.options[0]

        except:
            
            filePath.value = ''
            filesDropdown.value = sorted(fb.keys())[0]
            loadedText.value = "<span style='color:red'><b>An error has occurred, insert your file's path</b></span>"
            filesDropdown.value = filesDropdown.options[0]

In [29]:
loadButton.observe(openCSVFile)

In [30]:
blankChartText = widgets.HTML(
    value = "<h2>or create new blank chart:</h2> <hr>"    
)

blankChartButton = widgets.ToggleButton(
    description='New chart',
    disabled=False,
    button_style='warning', 
)

dataBlankName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

createdText = widgets.HTML(
    value = "",
)

topBlock.children = topBlock.children + (blankChartText, widgets.HBox([dataBlankName, blankChartButton,]), createdText)

In [31]:
def newBlankChart(change):
    if blankChartButton.value:
        blankChartButton.value = False
        df = pd.DataFrame([np.array([0.0, 0.0])], columns=['A', 'B'])
        db.addData(df, dataBlankName.value)
        dataBlankName.value = ''
        header.layout = {'display' : 'none'}
        createdText.value = "<span style='color:green'><b>Blank chart successfully created</b></span>."
        if len(dataDropdown.options) == 1:
            bodyBlock.children = [chartBlock, plotBlock]
            mainBlock.children = mainBlock.children + (bottomBlock,)
        liveUpdateCheckbox.value = True

In [32]:
blankChartButton.observe(newBlankChart, 'value')

In [33]:
bodyBlock = widgets.HBox([])

chartBlock = widgets.VBox([],layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))
plotBlock = widgets.VBox([],layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))

In [34]:
mainBlock.children = [topBlock, bodyBlock]

In [35]:
dataDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose your data: </h4>',
    style = style_1
)

deleteDataButton = widgets.ToggleButton(
    description = 'Delete data',
    button_style = '',
    style = style_1,
)

chartBlock.children = chartBlock.children + (widgets.HBox([dataDropdown, deleteDataButton]), )

In [36]:
def deleteData(change):
    if deleteDataButton.value:
        deleteDataButton.value = False
        name = dataDropdown.value
        index = 0
        
        for i in range(len(dataDropdown.options)):
            opt = dataDropdown.options[i]
            if opt == name:
                index = i
                break
        
        index = index - 1
        
        dataDropdown.value = dataDropdown.options[index]
        db.removeData(name)
        refreshDataDropdown()

In [37]:
deleteDataButton.observe(deleteData, 'value')

In [38]:
precisionDropdown = widgets.Dropdown(
    options = [i for i in range(0, 15)],
    value = 6,
    description='<h4>Number of decimals: </h4>',
    style = style_1
)

chartBlock.children = chartBlock.children + (precisionDropdown, spacerText)

In [39]:
def changePrecision(change):
    qGridCurrent.df = qGridCurrent.get_changed_df()
    qGridCurrent.precision = int(precisionDropdown.value)

In [40]:
precisionDropdown.observe(changePrecision, 'value')

In [41]:
chart = widgets.VBox([])
columnOptionsBox = widgets.VBox([])
rowOptionsBox = widgets.VBox([])

qGridCurrent = qgrid.QGridWidget(
    df = db.getData(dataDropdown.value),
    precision = 6,
    editable = True
)

columnOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

rowOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

In [42]:
def refreshColumnDropdowns():
    chooseColumnDropdown.options = qGridCurrent.df.columns.values
    chooseRenameColumnDropdown.options = qGridCurrent.df.columns.values
    finalColDropdown.options = qGridCurrent.df.columns.values
    aColDropdown.options = qGridCurrent.df.columns.values
    bColDropdown.options = qGridCurrent.df.columns.values

In [43]:
def refreshChart(change):
    
    old = change.get('old')
    new = change.get('new')

    if old != new and old != None:
        df = qGridCurrent.get_changed_df()
        db.updateData(df, old)
        
    qGridCurrent.df = db.getData(new)
    refreshColumnDropdowns()

In [44]:
dataDropdown.observe(refreshChart, 'value')

In [45]:
columnBox = widgets.VBox([])

newColumnName = widgets.Text(
    value='',
    placeholder="New column's name:",
    description='',
    disabled=False
)

addColumnButton = widgets.ToggleButton(
    description = 'Add column',
    button_style = '',
    style = style_1,
)

chooseColumnDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a column: </h4>',
    style = style_1
)

removeColumnButton = widgets.ToggleButton(
    description = 'Remove column',
    button_style = '',
    style = style_1,
)

renameColumnName = widgets.Text(
    value='',
    placeholder="Column's new name:",
    description='',
    disabled=False
)

renameColumnButton = widgets.ToggleButton(
    description = 'Rename column',
    button_style = '',
    style = style_1,
)

chooseRenameColumnDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a column: </h4>',
    style = style_1
)

columnBox.children = (widgets.HBox([
                          newColumnName,
                          addColumnButton]),
                      widgets.HTML(value = "<hr style = 'margin-top: 5px; margin-bottom: 0px;'>"),
                      widgets.HBox([
                          chooseColumnDropdown,
                          removeColumnButton]),
                      widgets.HTML(value = "<hr style = 'margin-top: 5px; margin-bottom: 0px;'>"),
                      chooseRenameColumnDropdown,
                      widgets.HBox([
                          renameColumnName,
                          renameColumnButton]),
                    )

In [46]:
def addColumn(change):
    if addColumnButton.value:
        addColumnButton.value = False
        name = newColumnName.value
        nameFound = False
        if name == '' or name in qGridCurrent.df.columns.values:
            alphabet0 = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
            alphabet = [elem for elem in alphabet0]
            counter = 0
            while nameFound == False:
                for elem in alphabet:
                    if elem not in list(qGridCurrent.df.columns.values):
                        name = elem
                        nameFound = True
                        break
                alphabet = [alphabet0[counter]+alphabet[i] for i in range(len(alphabet0))]
                counter = counter + 1
                
        qGridCurrent.df = qGridCurrent.get_changed_df()
        qGridCurrent.df[name] = 0.0
        qGridCurrent.df = qGridCurrent.df
        
        refreshColumnDropdowns()

        refreshPlotterSelectors(change)
        newColumnName.value = ''

In [47]:
addColumnButton.observe(addColumn, 'value')

In [48]:
def removeColumn(change):
    if removeColumnButton.value:
        removeColumnButton.value = False
        qGridCurrent.df = qGridCurrent.get_changed_df()
        qGridCurrent.df = qGridCurrent.df.drop(axis = 'columns', columns = chooseColumnDropdown.value)
        
        refreshColumnDropdowns()
        
        refreshPlotterSelectors(change)

In [49]:
removeColumnButton.observe(removeColumn, 'value')

In [50]:
def renameColumn(change):
    if renameColumnButton.value:
        renameColumnButton.value = False
        
        if renameColumnName.value not in chooseRenameColumnDropdown.options and renameColumnName.value != '':
            qGridCurrent.df = qGridCurrent.get_changed_df().rename(columns={ chooseRenameColumnDropdown.value : renameColumnName.value})
            qGridCurrent.df = qGridCurrent.df

            refreshColumnDropdowns()

            refreshPlotterSelectors(change)
            renameColumnName.value = ''

In [51]:
renameColumnButton.observe(renameColumn, 'value')

In [52]:
rowBox = widgets.VBox([])

addRowButton = widgets.ToggleButton(
    description = 'Add row',
    button_style = '',
    style = style_1,
)

removeRowButton = widgets.ToggleButton(
    description = 'Remove row',
    button_style = '',
    style = style_1,
)

rowBox.children = (widgets.HBox([
                          addRowButton,
                          removeRowButton], layout = widgets.Layout(align_self = 'center')),
                    )

In [53]:
def addRow(change):
    if addRowButton.value:
        addRowButton.value = False
        df = qGridCurrent.get_changed_df()
        df.loc[len(df.index)] = [0.0 for i in range(len(df.columns))]
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [54]:
addRowButton.observe(addRow, 'value')

In [55]:
def removeRow(change):
    if removeRowButton.value:
        removeRowButton.value = False
        df = qGridCurrent.get_changed_df()
        df = df.drop(axis = 'index', index=qGridCurrent.get_selected_rows())
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [56]:
removeRowButton.observe(removeRow, 'value')

In [57]:
exportAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%'),
    title = 'Export your chart'
)

exportAccordion.set_title(0, 'Export your chart')

exportBox = widgets.VBox([])

In [58]:
exportCSVBox = widgets.VBox()

exportCSVButton = widgets.ToggleButton(
    description = 'Export as CSV',
    button_style = '',
    style = style_1,
)

exportCSVName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

exportCSVText = widgets.HTML(
    value = "",
) 

exportCSVBox.children = (widgets.HBox([exportCSVName, exportCSVButton], layout = widgets.Layout(align_self = 'center')), exportCSVText)

In [59]:
def exportCSV(change):
    if exportCSVButton.value:
        exportCSVButton.value = False
        
        try:
            exportCSVText.value ="<b>Creating CSV file...</b>"
            filename = exportCSVName.value
            
            if filename == '':
                filename = dataDropdown.value.replace(' ', '_')
                
            if '.csv' not in filename:
                filename = filename + '.csv'
                
            file = qGridCurrent.get_changed_df().to_csv(filename, index=False)
            exportCSVText.value = "<span style='color:green'><b>CSV file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download CSV</button></form>"
            exportCSVName.value = ''
            refreshFilesDropdown()
        except:
            exportCSVText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            exportCSVName.value = ''

In [60]:
exportCSVButton.observe(exportCSV, 'value')

In [61]:
exportTexBox = widgets.VBox()

exportTexButton = widgets.ToggleButton(
    description = 'Export as TEX',
    button_style = '',
    style = style_1,
)

exportTexName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

exportTexText = widgets.HTML(
    value = "",
) 

exportTexBox.children = (widgets.HBox([exportTexName, exportTexButton], layout = widgets.Layout(align_self = 'center')), exportTexText)

exportAccordion.children = (exportBox,)
exportBox.children = (exportCSVBox, exportTexBox)

In [62]:
def exportTex(change):
    if exportTexButton.value:
        exportTexButton.value = False
        
        try:
            exportTexText.value ="<b>Creating TEX file...</b>"
            filename = exportTexName.value
            
            if filename == '':
                filename = dataDropdown.value.replace(' ', '_')
                
            if '.tex' not in filename:
                filename = filename + '.tex'
                
            file = qGridCurrent.get_changed_df().to_latex(filename, index=False)
            exportTexText.value = "<span style='color:green'><b>CSV file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download TEX</button></form>"
            exportTexName.value = ''
            refreshFilesDropdown()
        except:
            exportTexText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            exportTexName.value = ''

In [63]:
exportTexButton.observe(exportTex, 'value')

In [64]:
operationAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%'),
    title = 'Operate with your columns'
)

operationAccordion.set_title(0, 'Operate with the columns')

operationBox = widgets.VBox([])

operationText = widgets.HTML(
    value = "You can operate with the colums: <b>new column = (constant x column1 ^ constant) operation (constant x column2 ^ constant)</b>"    
)

finalColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

equals = widgets.HTMLMath(
    value = '\( = \)',
)

operationExpression = widgets.HTMLMath(
    value = '\( C1 = a \cdot C2^{b} * c \cdot C3^{d}\)',
)

ber = widgets.HTMLMath(
    value = '^',
)

const1 = widgets.FloatText(
    value=1,
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const2 = widgets.FloatText(
    value=1,
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const3 = widgets.FloatText(
    value=1,
    disabled=False,
    layout = widgets.Layout(width='50px'),
    description = ''
)

const4 = widgets.FloatText(
    value=1,
    disabled=False,
    layout = widgets.Layout(width='50px'),
    description = ''
)

dot = widgets.HTMLMath(
    value = '\( \cdot \)',
)

aColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

opColDropdown = widgets.Dropdown(
    options = ['+','-','x','/'],
    description='',
    layout = widgets.Layout(width='50px')
)

bColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

operateButton = widgets.ToggleButton(
    description='Calculate',
    disabled=False,
    button_style='',
    layout = widgets.Layout(align_self = 'center')
)

operationBox.children = (widgets.HBox([widgets.VBox([finalColDropdown]),
                         widgets.VBox([equals]),
                         widgets.VBox([const1,const2]),
                         widgets.VBox([dot,dot]),
                         widgets.VBox([aColDropdown,bColDropdown]),
                         widgets.VBox([ber,ber]),
                         widgets.VBox([const3,const4]),
                         widgets.VBox([opColDropdown])
                                      ]),
                         operateButton,
                        )

operationAccordion.children = (widgets.VBox([operationText, operationBox,]),)

In [65]:
def calculateColumn(change):
    if operateButton.value:
        operateButton.value = False
        
        op = opColDropdown.value
        c1 = const1.value
        c2 = const2.value
        c3 = const3.value
        c4 = const4.value
        a = np.array(qGridCurrent.get_changed_df()[aColDropdown.value])
        b = np.array(qGridCurrent.get_changed_df()[bColDropdown.value])

        with np.errstate(divide='ignore'):
            a = a**c3
            a[a == np.inf] = 0
            a = np.nan_to_num(a)

            b = b**c4
            b[b == np.inf] = 0
            b = np.nan_to_num(b)

        if op == '+':
            new = c1*a + c2*b

        elif op == '-':
            new = c1*a - c2*b

        elif op == 'x':
            new = (c1*a) * (c2*b)


        elif op == '/':

            with np.errstate(divide='ignore', invalid='ignore'):
                c = np.true_divide(c1*a,c2*b)
                c[c == np.inf] = 0
                new = np.nan_to_num(c)

        df = qGridCurrent.get_changed_df()
        df[finalColDropdown.value] = new
        
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [66]:
operateButton.observe(calculateColumn, 'value')

In [67]:
chartBlock.children = chartBlock.children + (columnOptionsAccordion, rowOptionsAccordion, qGridCurrent, operationAccordion, exportAccordion )

In [68]:
columnOptionsAccordion.children = (columnOptionsBox,)
columnOptionsAccordion.set_title(0, 'Edit columns')

columnOptionsBox.children = columnOptionsBox.children + (columnBox,)

In [69]:
rowOptionsAccordion.children = (rowOptionsBox,)
rowOptionsAccordion.set_title(0, 'Edit rows')

rowOptionsBox.children = rowOptionsBox.children + (rowBox,)

In [70]:
plotBox = widgets.VBox([])

In [71]:
plotterAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

plotterAccordion.set_title(0, 'Plot your data:')
plotter = widgets.VBox([])

plotterText = widgets.HTML(
    value = "<br><h2>Plot your data:</h2>"    
)

plotterSelectors = widgets.VBox([])

ySelector = widgets.Dropdown(
    options = list(qGridCurrent.df),
    description='<h6>Y axis: </h6>',
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

xSelector = widgets.Dropdown(
    options = list(qGridCurrent.df),
    description='<h6>X axis: </h6>',
    rows = 10,
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

pointsColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#7800c8',
    disabled=False,
    layout = widgets.Layout(width = '200px')
)


replaceButton = widgets.ToggleButton(
    description = 'Replace plot',
    button_style = 'danger',
    style = style_1,
)

addButton = widgets.ToggleButton(
    description = 'Add to plot',
    button_style = 'warning',
    style = style_1,
)

liveUpdateCheckbox = widgets.Checkbox(
    value=True,
    description='Live update (If checked, changes in the data chart will update the plot) (experimental)',
    description_tooltip = 'If checked, changes in the data chart will update the plot (experimental)'
)

                                
plotterSelectors.children = [ widgets.HBox([xSelector, ySelector, pointsColor]),
                              widgets.HBox([replaceButton, addButton], layout = widgets.Layout(align_self = 'center'))]

plotter.children = [plotterSelectors, liveUpdateCheckbox]
plotterAccordion.children = [plotter]

In [72]:
liveUpdateCheckbox.description_tooltip = 'fidsaof`dsao`fds'

In [73]:
plotBlock.children = plotBlock.children + (plotterText, plotter,)

In [74]:
scaleX = bqm.LinearScale(min = 0, max = 0)
scaleY = bqm.LinearScale(min = 0, max = 0)

axisX = bqa.Axis(scale=scaleX,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 5,
                grid_lines = 'dashed',
                grid_color = '#8e8e8e', 
                label='x',
                label_location='middle',
                label_offset='0.5em')

axisY = bqa.Axis(scale=scaleY,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 4,
                grid_lines = 'dashed',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='y',
                label_location='middle',
                label_offset='50px')

tooltipScat = bq.Tooltip(fields=['x', 'y'], labels=[ 'x', 'y'], formats=['4.2f', '4.2f'])

In [75]:
scat = bqm.Scatter(x = np.array([]), 
                   y = np.array([]), 
                   scales = {'x': scaleX, 'y': scaleY}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   selected_style={'opacity': '1'},
                   unselected_style={'opacity': '0.2'},
                   )

In [76]:
lin = bqm.Lines(x = np.array([]), 
                y = np.array([]), 
                scales = {'x': scaleX, 'y': scaleY}, 
                visible = False,
                colors = ['#fad87c']
                )

In [77]:
def refreshPlotterSelectors(change):
    xSelector.options = list(qGridCurrent.df)
    ySelector.options = list(qGridCurrent.df)

In [78]:
dataDropdown.observe(refreshPlotterSelectors, 'value')

In [79]:
def refreshDimensions(dataX, dataY, scatter=scat, line=lin):
    
    index = []
    
    for i in range(len(dataX)):
        if isNotNumber(dataX[i]) or isNotNumber(dataY[i]):
            index.append(i)
 
    dataX = np.delete(dataX, index).astype(np.float)
    dataY = np.delete(dataY, index).astype(np.float)
    
    dataMinX = min(dataX)
    dataMaxX = max(dataX)
    
    dataMinY = min(dataY)
    dataMaxY = max(dataY)
    
    dataWidth  = dataMaxX - dataMinX
    dataHeight = dataMaxY - dataMinY
    
    if dataWidth == 0.0:
        dataWidth = 10.0
    
    if dataHeight == 0.0:
        dataHeight = 10.0
    
    scaleX = bqm.LinearScale(min = dataMinX - 0.1*dataWidth, max = dataMaxX + 0.1*dataWidth)
    scaleY = bqm.LinearScale(min = dataMinY - 0.1*dataHeight, max = dataMaxY + 0.1*dataHeight)

    axisX.scale = scaleX
    axisY.scale = scaleY
    
    scatter.scales = {'x': scaleX, 'y': scaleY}
    line.scales = {'x': scaleX, 'y': scaleY}

    pz.scales = {'x': [scaleX], 'y': [scaleY]}
    pzx.scales = {'x': [scaleX]}
    pzy.scales = {'y': [scaleY]}

    squareSel.x_scale = scaleX
    squareSel.y_scale = scaleY
        
    sim_pz.scales = {'x': [scaleX], 'y': [scaleY]}
    sim_pzx.scales = {'x': [scaleX]}
    sim_pzy.scales = {'y': [scaleY]}

    sim_squareSel.x_scale = scaleX
    sim_squareSel.y_scale = scaleY

In [80]:
def replacePlot(change):
    
    replaceButton.value = False
    
    addPointsButton.value = False
    removePointsButton.value = False

    data = db.getData(dataDropdown.value)

    dataX = qGridCurrent.get_changed_df().get(xSelector.value).values 
    dataY = qGridCurrent.get_changed_df().get(ySelector.value).values 

    if dataX is not None and dataY is not None:

        if len(dataX) > 1:

            refreshDimensions(dataX, dataY)

            scat.observe(null)

            [scat.x, scat.y] = dataX, dataY
            scat.colors = [pointsColor.value for elem in dataX]
            
            scat.selected = None
            
            showRegressionCheckbox.value = False
            showEstimatedCheckbox.value = False
            showDistancesCheckbox.value = False

            lin.x = np.array([])
            lin.y = np.array([])

            axisX.label = xSelector.value
            axisY.label = ySelector.value
            
            mainPlot.marks = [dist, scatEstimated, lin,scat]
            mainPlot.axes = ([axisX, axisY])
            mainPlot.title = ''
            
            w = widgets.VBox(children = [widgets.HTML(value='<br/>'), zoom_interacts, compressZoomButton, ResetZoomButton], layout = widgets.Layout(width='10%'))
            v = widgets.HBox([mainPlot, w])
            
            mainPlotBox.children = (v,                                    
                                    widgets.HBox([simulationName, simulateButton, changeViewButton]))
            
            plotBlock.children = [plotterText, plotter, plotOptionsAccordion, mainPlotBox, regressionOptionsAccordion, exportPlotAccordion]

In [81]:
replaceButton.observe(replacePlot)

In [82]:
def addToPlot(change):
    
    if addButton.value:
        addButton.value = False

        addPointsButton.value = False
        removePointsButton.value = False

        data = db.getData(dataDropdown.value)
  
        dataX = np.append(data.get(xSelector.value).values, scat.x)
        dataY = np.append(data.get(ySelector.value).values, scat.y)

        colors = scat.colors
        
        if len(scat.x) != 0 and len(scat.y) != 0:

            refreshDimensions(dataX, dataY)

            for i in range(len(data.get(xSelector.value))):
                colors.append(pointsColor.value)

            scat.colors = colors    
                
            [scat.x, scat.y] = dataX, dataY

            scat.selected = None

            updateRegression(change)
            updateDistances(change)
            
        else:            
            replacePlot(change)

In [83]:
addButton.observe(addToPlot)

In [84]:
mainPlot = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(width='80%', align_self='center'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=75, right=20)
               )

In [85]:
mainPlotBox = widgets.VBox([])

In [86]:
mainPlotBox.display = False

In [87]:
def null(change):
    pass

In [88]:
pz = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                          scales={'x': [mainPlot.scale_x], 'y': [mainPlot.scale_y]})

pzx = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                           scales={'x': [mainPlot.scale_x]})

pzy = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                           scales={'y': [mainPlot.scale_y], })

squareSel = bq.interacts.BrushSelector(marks=[scat], x_scale=mainPlot.scale_x, y_scale=mainPlot.scale_y)
handSel = bq.interacts.LassoSelector(marks=[scat])

#icons from https://fontawesome.com/v4.7.0/icons/
zoom_interacts = widgets.ToggleButtons(
                                        value = None,
                                        options=[
                                            ('xy ', pz), 
                                            ('x ', pzx), 
                                            ('y ', pzy),  
                                            (' ', squareSel),
                                            (' ', handSel),
                                            (' ', None),],
                                            icons = ["arrows", "arrows-h", "arrows-v", "pencil-square-o", "pencil", "stop"],
                                            tooltips = ["zoom/pan in x & y",
                                                        "zoom/pan in x only",
                                                        "zoom/pan in y only",
                                                        "square selector",
                                                        "lasso selector",
                                                        "cancel interaction"],
                                        layout = widgets.Layout(
                                            width = '50px',
                                            margin_left = '0px',
                                            margin_right = '0px'
                                        ),
                                        style = {'button_width' : '50px'}
                                        )

In [89]:
link((zoom_interacts, 'value'), (mainPlot, 'interaction'));

In [90]:
ResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reset zoom',
    icon='arrows-alt',
    layout = widgets.Layout(width='50px')
)

In [91]:
def resetZoom(new):
    refreshDimensions(scat.x, scat.y)
    updateDistances(None)

In [92]:
ResetZoomButton.on_click(resetZoom)

In [93]:
compressZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Zoom to selected area',
    icon='compress',
    layout = widgets.Layout(width='50px')
)

In [94]:
def compressZoom(new):
    if scat.selected != None:
        if len(scat.selected) > 2:
            dataX = np.take(scat.x, scat.selected)
            dataY = np.take(scat.y, scat.selected)   
            refreshDimensions(dataX, dataY)
            updateDistances(None)
            zoom_interacts.value = None

In [95]:
compressZoomButton.on_click(compressZoom)

In [96]:
plotOptionsBox = widgets.VBox([])

plotOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)


plotOptionsAccordion.set_title(0, 'Plot options')
plotOptionsAccordion.children = (plotOptionsBox, )

In [97]:
checkboxes = widgets.HBox([], layout = widgets.Layout(align_self = 'center', width = '100%'))

grid = widgets.Checkbox(
    value=False,
    description='Show grid',
)

legendBold = widgets.Checkbox(
    value=False,
    description='Bold legend',
)

numbersBold = widgets.Checkbox(
    value=False,
    description='Bold numbers',
)

checkboxes.children = (grid, numbersBold, )
plotOptionsBox.children = plotOptionsBox.children + (checkboxes,)

In [98]:
def updatePlotCheckboxes(change):
    
    if grid.value:
        axisX.grid_lines = 'dashed'
        axisY.grid_lines = 'dashed'
        
    else:
        axisX.grid_lines = 'none'
        axisY.grid_lines = 'none'
        
    if numbersBold.value:
        axisX.tick_style={'stroke': 'black', 'font-size': '15px'}
        axisY.tick_style={'stroke': 'black', 'font-size': '15px'}
        
    else:
        axisX.tick_style={'stroke': 'none', 'font-size': '15px'}
        axisY.tick_style={'stroke': 'none', 'font-size': '15px'}

In [99]:
grid.observe(updatePlotCheckboxes, 'value')
numbersBold.observe(updatePlotCheckboxes, 'value')

In [100]:
renamePlotBox = widgets.HBox([])

plotTitle = widgets.Text(
    value= '',
    placeholder="Type plot's new title",
    description='Plot name:',
    disabled=False
)

renamePlotButton = widgets.ToggleButton(
    description = 'Rename plot',
    button_style = 'warning',
    style = style_1,
)

renamePlotBox.children = (plotTitle, renamePlotButton)
plotOptionsBox.children = plotOptionsBox.children + (renamePlotBox,)

In [101]:
def renamePlot(change):
    
    if renamePlotButton.value:
        renamePlotButton.value = False
        mainPlot.title = str(plotTitle.value)
        plotTitle.value = ''

In [102]:
renamePlotButton.observe(renamePlot, 'value')

In [103]:
renameAxisBox = widgets.VBox([])

xName = widgets.Text(
    value='',
    placeholder="Type X axis' new title",
    description='X axis:',
    disabled=False
)

renameXAxis = widgets.ToggleButton(
    description = 'Rename X',
    button_style = 'warning',
    style = style_1,
)

yName = widgets.Text(
    value='',
    placeholder="Type Y axis' new title",
    description='Y axis:',
    disabled=False
)

renameYAxis = widgets.ToggleButton(
    description = 'Rename Y',
    button_style = 'warning',
    style = style_1,
)

renameAxisBox.children = (widgets.HBox([xName, renameXAxis]),
                          widgets.HBox([yName, renameYAxis]))

plotOptionsBox.children = plotOptionsBox.children + (renameAxisBox,)

In [104]:
def renameAxis(change):
    
    if renameXAxis.value:
        renameXAxis.value = False
        axisX.label = xName.value
        xName.value = ''
        
    if renameYAxis.value:
        renameYAxis.value = False
        axisY.label = yName.value   
        yName.value = ''

In [105]:
renameXAxis.observe(renameAxis, 'value')
renameYAxis.observe(renameAxis, 'value')

In [106]:
regressionOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)


regressionOptionsAccordion.set_title(0, 'Regression')

In [107]:
regressionOptionsBox = widgets.VBox([])

regressionBox = widgets.VBox([])

showRegressionCheckbox = widgets.Checkbox(
    value=False,
    description='Show regression',
    layout = widgets.Layout(width = '200px')
)

regressionColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#fad87c',
    disabled=False
)

regressionOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

regressionBox.children = (widgets.HTML(value='<h4>Edit regression:</h4>'), widgets.HBox([regressionColor, regressionOpacity]))

regressionOptionsBox.children = (showRegressionCheckbox,)
regressionOptionsAccordion.children = (regressionOptionsBox,)

In [108]:
def showRegression(change):
    
    if showRegressionCheckbox.value:
        lin.visible = True
        updateRegression(change)
        updateDistances(change)
        
    else:
        lin.visible = False

In [109]:
showRegressionCheckbox.observe(showRegression, 'value')

In [110]:
def changeRegressionColor(change):
    lin.colors = [regressionColor.value]

In [111]:
regressionColor.observe(changeRegressionColor, 'value')

In [112]:
def changeRegressionOpacity(change):
    lin.opacities = [regressionOpacity.value]

In [113]:
regressionOpacity.observe(changeRegressionOpacity, 'value')

In [114]:
regressionDecimals = widgets.Dropdown(
    options = [i for i in range(0,14)],
    value = 2,
    description='<h4>Number of decimals: </h4>',
    style = style_1
)

In [115]:
regressionOrder = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=5.0,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(width = '200px')
)

regressionErrorText = widgets.HTML(
    value = "",
)

regressionValues = widgets.VBox()

In [116]:
def updateRegression(change):

    if regressionExpression not in regressionOptionsBox.children:
    
        regressionOptionsBox.children = [widgets.HBox([showRegressionCheckbox, widgets.Label('Order of regression'),regressionOrder,]),
                                        regressionExpression,
                                        regressionValues,
                                        regressionDecimals,
                                        widgets.HTML(value='<br>'),
                                        widgets.HBox([showDistancesCheckbox,showEstimatedCheckbox]),
                                        widgets.HTML(value='<hr>'),
                                        regressionBox,
                                        distancesBox,
                                        estimatedBox
                                        ]

    if scat.selected == None:
        
        dataX = scat.x
        dataY = scat.y
        
    else:
        
        if len(scat.selected) < 3:
        
            dataX = scat.x
            dataY = scat.y
        
        else:
            
            dataX = np.take(scat.x, scat.selected)
            dataY = np.take(scat.y, scat.selected)    
            
    lin.x = np.linspace(start=np.min(dataX), stop=np.max(dataX), num=1000)
    
    try:
        if len(dataX) > regressionOrder.value + 1:
            poly, cov = np.polyfit(dataX, dataY, regressionOrder.value, cov=True)
            regressionErrorText.value = ''

        else:
            poly = np.polyfit(dataX, dataY, len(dataX) - 1)
            cov = np.zeros([len(poly),len(poly)])
            
            if regressionOrder.value >= len(dataX):
                regressionErrorText.value = "<span style='color:red'><b>You need more points for a regression of order " + str(regressionOrder.value) + "</b></span>"
            else:
                regressionErrorText.value = ''
    except:
        return None
    lin.y = np.polyval(poly, lin.x)
    
    
    if squareSel.brushing == False:
    
        expression = '\(y = \;\)'
        maxPower = len(poly)-1

        decimals = regressionDecimals.value
        chars = 'abcdef'

        errors = np.sqrt(np.diag(cov))

        regressionValues.children = []

        for i in range(len(poly)):

            coef = np.around([ poly[i],],decimals)[0]

            error = np.around([errors[i],], decimals)[0]

            power = maxPower-i

            if power == 1:

                expression = expression + '\(' + chars[i] + '\; x \; + \; \)' 

            elif power == 0:
                expression = expression + '\(' + chars[i] + ' \)'

            else:
                expression = expression + '\(' + chars[i] + '\; x^{' + str(power) + '} \; + \; \)'

            s = '\(' + chars[i] + '\; = \; ' + str(coef) + '\; \pm \;' + str(error) + '\)'
            regressionValues.children = regressionValues.children + (widgets.HTMLMath(value = s , description = ''), )

        regressionExpression.value = expression
        errors = np.sqrt(np.diag(cov))
        errorsExp = ''

        for error in errors:
            error = np.around([error,], decimals)[0]
            errorsExp = errorsExp + '\(' + str(error) + '\; \; \; \)'

        errorsExpression.value = errorsExp    

    updateDistances(change)

In [117]:
regressionOrder.observe(updateRegression, 'value')

In [118]:
regressionDecimals.observe(updateRegression, 'value')

In [119]:
scat.observe(updateRegression, 'selected')

In [120]:
squareSel.observe(updateRegression, 'brushing')

In [121]:
regressionExpression = widgets.HTMLMath(
    value = '\(y =\)',
    description = 'Expression:'
)

In [122]:
errorsExpression = widgets.HTMLMath(
    value = '\(',
    description = 'Errors:'
)

In [123]:
simulationName = widgets.Text(
    value='',
    placeholder="Type your simulation's name (optional):",
    disabled = False,
)

simulateButton = widgets.ToggleButton(
    description = 'Simulate',
    button_style = '',
    style = style_1,
)

In [124]:
def simulate(change):
    if simulateButton.value:
        simulateButton.value = False
        pb.addSimulation(mainPlot, simulationName.value)
        pb.addPlot(mainPlot, simulationName.value)
        refreshPlotDropdown()
        refreshSimDropdown()
        changeViewButton.value = True
        simulationName.value = ''

In [125]:
simulateButton.observe(simulate,'value')

In [126]:
pngName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

savePNGButton = widgets.ToggleButton(
    description = 'Save as PNG',
    button_style = '',
    style = style_1,
)

pngCreatedText = widgets.HTML(
    value = "",
    ) 

pdfName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

savePDFButton = widgets.ToggleButton(
    description = 'Save as PDF',
    button_style = '',
    style = style_1,
)
    
pdfCreatedText = widgets.HTML(
    value = "",
    ) 

exportPlotAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%'),
    title = 'Export your chart'
)

exportPlotAccordion.set_title(0, 'Export your plot')

exportPlotBox = widgets.VBox()
exportPlotBox.children = [widgets.HBox([pdfName,savePDFButton], layout = widgets.Layout(align_self = 'center')),
                        widgets.HBox([pdfCreatedText]),
                        widgets.HBox([pngName,savePNGButton], layout = widgets.Layout(align_self = 'center')),
                        widgets.HBox([pngCreatedText]),]

exportPlotAccordion.children = (exportPlotBox,)

In [127]:
def matplotlib_figure(pdf, png, fig=mainPlot):

    plt.ioff()
    
    s = fig.marks[3]
    l = fig.marks[2]
    
    ax = fig.axes[0]
    ay = fig.axes[1]
    
    # irudiaren diseinuerako
    plt.rc('text', usetex=True)
    mpl.rcParams['errorbar.capsize'] = 3
    colors_ = list(mcolors.TABLEAU_COLORS.values())
       
    # irudiko eskalak definitzeko: hobetu egin behar da, balioen eta tarteen arabera egokiak diren tick-en kopurua finkatzeko, beti kopuru txikia izanik  
    
    if fig.marks[3].selected == None:
        
        dataX = fig.marks[3].x
        dataY = fig.marks[3].y
        
    else:
        
        if len(fig.marks[3].selected) < 3:
        
            dataX = fig.marks[3].x
            dataY = fig.marks[3].y
        
        else:
            
            dataX = np.take(fig.marks[3].x, fig.marks[3].selected)
            dataY = np.take(fig.marks[3].y, fig.marks[3].selected)    

    
    x_min, x_max, y_min, y_max = min(dataX), max(dataX), min(dataY), max(dataY)

    # irudia
    fig_pdf, ax0 = plt.subplots(nrows=1, ncols=1, figsize=(16, 12), );
    plt.subplots_adjust(left=0.25, bottom=0.25, right=0.9, top=None, wspace=0.0, hspace=0.0)
    ax0.tick_params(axis='both', labelsize=30, pad=10, length=12)
   
    # limeteak
    
    dataWidth = x_max - x_min
    dataHeight = y_max - y_min
    
    ax0.set_xlim(x_min-dataWidth*0.1, x_max+dataWidth*0.1)
    ax0.set_ylim(y_min-dataHeight*0.1, y_max+dataHeight*0.1)

    # ticks
    ax0.yaxis.set_major_locator(mpl.ticker.NullLocator());
    ax0.xaxis.set_major_locator(mpl.ticker.NullLocator());
    
    #plt.xticks()
    
    ax0.yaxis.set_major_locator(plt.MaxNLocator(nbins=ay.num_ticks));
    ax0.xaxis.set_major_locator(plt.MaxNLocator(nbins=ax.num_ticks));

    # etiketak
    if len(fig.title.split()) > 0: 
        ax0.set_title(r'\textrm{%s}' % fig.title, size=35, pad=20)
    ax0.set_xlabel(r'\textrm{%s}' % ax.label, size=30, labelpad=15)
    ax0.set_ylabel(r'\textrm{%s}' % ay.label, size=30, labelpad=15)

    # Legendan agertuko dena, defektuz finkoa da, baina beti aldatu daiteke
    labels = [r'\textrm{Data}', r'$\textrm{Regression}$']
   
    # datuak irudikatu
    ax0.scatter(dataX, dataY, label=labels[0], color=fig.marks[3].colors[0], s=180);
    
    order = [0]
    
    if showRegressionCheckbox.value:
        ax0.plot(fig.marks[2].x, fig.marks[2].y, label=labels[1], color=fig.marks[2].colors[0], lw=5, alpha=0.5);
        order.append(1)
        
    # egokia den ordenean legendakoak adierazteko
    handles, labels = plt.gca().get_legend_handles_labels()

    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
    ax0.legend(bbox_transform=ax0.transAxes,
               #bbox_to_anchor=(0.95, 0.30),
               loc='best',
               ncol=1,
               borderaxespad=1.5,
               frameon=False,
               fontsize=25);
   
    # doiketaren emaitzak irudian bertan idazteko
    if showRegressionCheckbox.value:
        fig_pdf.text(0.5, -0.25, regressionExpression.value, size=25, horizontalalignment='center', rotation='horizontal', transform=ax0.transAxes);
        
        space = -0.35
        for error in regressionValues.children:
            fig_pdf.text(0.5, space, error.value, size=25, horizontalalignment='center', rotation='horizontal', transform=ax0.transAxes);
            space = space - 0.075
            
    # irudia gorde
    if pdf:
        save_filename = pdfName.value

        if len(save_filename.split()) == 0:
            save_filename = 'pdf_file'

        if '.pdf' not in save_filename:
            save_filename = save_filename + '.pdf'

        plt.savefig(save_filename, format='pdf', dpi=150, bbox_inches="tight");
        return (save_filename)
    
    if png:
        save_filename = pngName.value
        if len(save_filename.split()) == 0:
            save_filename = 'png_file'

        if '.png' not in save_filename:
            save_filename = save_filename + '.png'
        plt.savefig(save_filename, format='png', dpi=150, bbox_inches="tight");
        return (save_filename)

In [128]:
def savePDF(change):
    if savePDFButton.value:
        savePDFButton.value = False
        
        try:
            pdfCreatedText.value ="<b>Creating PDF file...</b>"
            filename = matplotlib_figure(pdf = True, png = False, fig = mainPlot)
            pdfCreatedText.value = "<span style='color:green'><b>PDF file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download PDF</button></form>"
            pdfName.value = ''

        except:
            pdfCreatedText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            pdfName.value = ''

In [129]:
savePDFButton.observe(savePDF, 'value')

In [130]:
def savePNG(change):
    if savePNGButton.value:
        savePNGButton.value = False
        
        try:
            pngCreatedText.value ="<b>Creating PNG file...</b>"
            filename = matplotlib_figure(pdf = False, png = True, fig = mainPlot)
            pngCreatedText.value = "<span style='color:green'><b>PNG file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download PNG</button></form>"
            pngName.value = ''

        except:
            pngCreatedText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            pngName.value = ''

In [131]:
savePNGButton.observe(savePNG, 'value')

In [132]:
scatEstimated = bqm.Scatter(x = np.array([]),
                             y = np.array([]),
                             scales = {'x': scaleX, 'y': scaleY},
                             enable_move = False,
                             visible = False,
                            colors = ['#f87878']
                            )

In [133]:
dist = bqm.Lines(x = np.array([]),
                 y = np.array([]),
                 scales = {'x': scaleX, 'y': scaleY},
                 line_style = 'dashed',
                 visible = False,
                 colors = ['#cdb4c8'],
                 )

In [134]:
def updateDistances(change):

    scaleX = scat.scales.get('x')
    scaleY = scat.scales.get('y')
    
    dist.scales = {'x': scaleX, 'y': scaleY}
    scatEstimated.scales = {'x': scaleX, 'y': scaleY}
    
    x = scat.x
    y = scat.y
    
    try:
        yInterp = np.interp(x, lin.x, lin.y) 
    
    except:
        return None
    
    scatEstimated.x = x
    scatEstimated.y = yInterp    
    
    xVal = []
    yVal = []

    for i in range(len(x)):
        for j in range(2):
            xVal.append(x[i])
        yVal.append(min(y[i],yInterp[i]))
        yVal.append(max(y[i],yInterp[i]))
        
    xDef = []
    yDef = []

    for i in range(0, len(xVal), 2):
        xDef.append(np.array([xVal[i], xVal[i+1]]))
        yDef.append(np.array([yVal[i], yVal[i+1]]))
        
    dist.x = xDef
    dist.y = yDef

In [135]:
distancesBox = widgets.VBox([])

showDistancesCheckbox = widgets.Checkbox(
    value=False,
    description='Show distances',
)

distancesColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#cdb4c8',
    disabled=False,
)

distancesOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='350px')
)

distancesBox.children = (widgets.HTML(value='<h4>Edit distances:</h4>'), widgets.HBox([distancesColor, distancesOpacity]))

In [136]:
def showDistances(change):
    
    if showDistancesCheckbox.value:
        dist.visible = True
        updateRegression(change)
        
    else:
        dist.visible = False

In [137]:
showDistancesCheckbox.observe(showDistances, 'value')

In [138]:
def changeDistancesColor(change):
    dist.colors = [distancesColor.value for elem in dist.x]

In [139]:
distancesColor.observe(changeDistancesColor, 'value')

In [140]:
def changeDistancesOpacity(change):
    dist.opacities = [distancesOpacity.value for elem in dist.x]

In [141]:
distancesOpacity.observe(changeDistancesOpacity, 'value')

In [142]:
estimatedBox = widgets.VBox([])

showEstimatedCheckbox = widgets.Checkbox(
    value=False,
    description='Show estimated points',
)

estimatedColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#f87878',
    disabled=False
)

estimatedOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='350px')
)

estimatedBox.children = (widgets.HTML(value='<h4>Edit estimated points:</h4>'), widgets.HBox([estimatedColor, estimatedOpacity]))

In [143]:
def showEstimated(change):
    
    if showEstimatedCheckbox.value:
        scatEstimated.visible = True
        updateRegression(change)
        
    else:
        scatEstimated.visible = False

In [144]:
showEstimatedCheckbox.observe(showEstimated, 'value')

In [145]:
def changeEstimatedColor(change):
    scatEstimated.colors = [estimatedColor.value]

In [146]:
estimatedColor.observe(changeEstimatedColor, 'value')

In [147]:
def changeEstimatedOpacity(change):
    scatEstimated.default_opacities = [estimatedOpacity.value]

In [148]:
estimatedOpacity.observe(changeEstimatedOpacity, 'value')

In [149]:
def update_plot_old(event, widget): 

    df = qGridCurrent.get_changed_df()

    old = event.get('old')
    index = event.get('index')
    new = event.get('new')

    if index < len(scat.x):
        try:
            dataX = scat.x
            dataY = scat.y    

            if old in dataX:
                scat.x = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
                scatIndex = np.where(dataX == old)[0]
                row = df.iloc[index].values

                indexes = [index for index in scatIndex if dataY[index] in row]

                for i in indexes:
                    dataX[i] = new                
                scat.x = dataX.tolist()


            elif old in dataY:
                scat.y = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
                scatIndex = np.where(dataY == old)[0]
                row = df.iloc[index].values
                indexes = [index for index in scatIndex if dataX[index] in row][0]
                dataY[indexes] = new                
                scat.y = dataY.tolist()

            refreshDimensions(dataX, dataY)
            updateRegression(None)
            updateDistances(None)

        except:
            return None     

In [150]:
def update_plot(event, widget): 

    
    if liveUpdateCheckbox.value:
        
        replacePlot(None)       

In [151]:
qGridCurrent.on(names = ['cell_edited'], handler = update_plot)

In [152]:
bottomBlock = widgets.HBox([])

In [153]:
changeViewButton = widgets.ToggleButton(
    disabled=False,
    button_style='',
    description='See simulations',
)

In [154]:
def changeView(change):
    
    if changeViewButton.value:
        changeViewButton.value = False 
        
        if len(plotDropdown.options) > 0:
            description = changeViewButton.description

            if description == 'See simulations':
                changeViewButton.description = 'See data'
                zoom_interacts.value = None
                bodyBlock.children = [savedBlock, simBlock]
                refreshSimDropdown()
                refreshPlotDropdown()
                refreshDimensions(simScat.x, simScat.y, simScat, simLin)
                simShowRegressionCheckbox.value = showRegressionCheckbox.value
                changeViewButton.layout = widgets.Layout(align_self = 'center')
                
            else:
                changeViewButton.description = 'See simulations'
                changeViewButton.layout = widgets.Layout()
                bodyBlock.children = [chartBlock, plotBlock]
                refreshDimensions(scat.x, scat.y)
                sim_zoom_interacts.value = None

In [155]:
changeViewButton.observe(changeView, 'value')

In [156]:
simBlock = widgets.VBox([], layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))

In [157]:
simDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose your simulations: </h4>',
    style = {'description_width': 'initial'},
    layout = widgets.Layout(width = '100%')
)

In [158]:
def refreshSimDropdown():
    simDropdown.options = pb.getSimulations()

In [159]:
def changeSim(change):
    name = simDropdown.value
    figure = pb.getPlot(name)
    addPointsButton.value = False
    dragXButton.value = False
    dragYButton.value = False
    
    simPlot.title = figure.title

    d = figure.marks[0] #distances
    se = figure.marks[1] #scatestimated
    l = figure.marks[2] #lines
    s = figure.marks[3] #scat
    
    simScat.x, simScat.y = s.x, s.y
    simScatEstimated.x, simScatEstimated.y = se.x, se.y
    simLin.x, simLin.y = l.x, l.y
    simDist.x, simDist.y = d.x, d.y
    
    sc_x = s.scales.get('x')
    sc_y = s.scales.get('y')
    
    simScat.scales = {'x': sc_x, 'y': sc_y}
    simScatEstimated.scales = {'x': sc_x, 'y': sc_y}
    simLin.scales = {'x': sc_x, 'y': sc_y}
    simDist.scales = {'x': sc_x, 'y': sc_y}
    
    simScat.colors = s.colors
    simScatEstimated.colors = se.colors
    simLin.colors = l.colors
    simDist.colors = d.colors
    
    simScat.visible = s.visible
    simScatEstimated.visible = se.visible
    simLin.visible = l.visible
    simDist.visible = d.visible
    
    simScat.selected = None

    simPlot.axes[0].label = figure.axes[0].label
    simPlot.axes[1].label = figure.axes[1].label
    refreshDimensions(simScat.x, simScat.y, simScat, simLin)
    simShowRegressionCheckbox.value = False
    simShowEstimatedCheckbox.value = False
    simShowDistancesCheckbox.value = False
    simUpdateRegression(None)
    simUpdateDistances(None)

In [160]:
simDropdown.observe(changeSim, 'value')

In [161]:
deleteSimButton = widgets.ToggleButton(
    description = 'Delete sim',
    button_style = '',
    style = style_1,
)

In [162]:
def deleteSim(change):
    if deleteSimButton.value:
        deleteSimButton.value = False
        
        name = simDropdown.value
        index = 0
        
        for i in range(len(simDropdown.options)):
            opt = simDropdown.options[i]
            if opt == name:
                index = i
                break
        
        index = index - 1
        
        simDropdown.value = simDropdown.options[index]
        pb.removeSimulation(name)
        
        refreshSimDropdown()

In [163]:
deleteSimButton.observe(deleteSim, 'value')

In [164]:
scaleX = bqm.LinearScale(min = 0, max = 0)
scaleY = bqm.LinearScale(min = 0, max = 0)

axisX = bqa.Axis(scale=scaleX,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='x',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axisY = bqa.Axis(scale=scaleY,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='y',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

tooltipScat = bq.Tooltip(fields=['x', 'y'], labels=[ 'x', 'y'], formats=['4.2f', '4.2f'])

simScat = bqm.Scatter(x = np.array([]), 
                   y = np.array([]), 
                   scales = {'x': scaleX, 'y': scaleY}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   selected_style={'opacity': '1'},
                   unselected_style={'opacity': '0.2'},
                   )

simLin = bqm.Lines(x = np.array([]), 
                y = np.array([]), 
                scales = {'x': scaleX, 'y': scaleY}, 
                visible = False,
                colors = ['#fad87c']
                )

simScatEstimated = bqm.Scatter(x = np.array([]),
                             y = np.array([]),
                             scales = {'x': scaleX, 'y': scaleY},
                             enable_move = False,
                             visible = False,
                            colors = ['#f87878']
                            )

simDist = bqm.Lines(x = np.array([]),
                 y = np.array([]),
                 scales = {'x': scaleX, 'y': scaleY},
                 line_style = 'dashed',
                 visible = False,
                 colors = ['#cdb4c8'],
                 )

In [165]:
simPlot = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(width='80%', align_self='center'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=75, right=20)
               )

simPlot.marks = [simDist, simScatEstimated, simLin, simScat]
simPlot.axes = ([axisX, axisY])

simPlotBox = widgets.VBox([])

In [166]:
sim_pz = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                          scales={'x': [simPlot.scale_x], 'y': [simPlot.scale_y]})

sim_pzx = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                           scales={'x': [simPlot.scale_x]})

sim_pzy = bq.interacts.PanZoom(allow_pan = True,
                          allow_zoom = True,
                           scales={'y': [simPlot.scale_y], })

sim_squareSel = bq.interacts.BrushSelector(marks=[simScat], x_scale=simScat.scales['x'], y_scale=simScat.scales['y'])
sim_handSel = bq.interacts.LassoSelector(marks=[simScat])

#icons from https://fontawesome.com/v4.7.0/icons/
sim_zoom_interacts = widgets.ToggleButtons(
                                        value = None,
                                        options=[
                                            ('xy ', sim_pz), 
                                            ('x ', sim_pzx), 
                                            ('y ', sim_pzy),  
                                            (' ', sim_squareSel),
                                            (' ', sim_handSel),
                                            (' ', None),],
                                            icons = ["arrows", "arrows-h", "arrows-v", "pencil-square-o", "pencil", "stop"],
                                            tooltips = ["zoom/pan in x & y",
                                                        "zoom/pan in x only",
                                                        "zoom/pan in y only",
                                                        "square selector",
                                                        "lasso selector",
                                                        "cancel interaction"],
                                        layout = widgets.Layout(
                                            width = '50px',
                                            margin_left = '0px',
                                            margin_right = '0px'
                                        ),
                                        style = {'button_width' : '50px'}
                                        )

In [167]:
link((sim_zoom_interacts, 'value'), (simPlot, 'interaction'));

In [168]:
simResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reset zoom',
    icon='arrows-alt',
    layout = widgets.Layout(width='50px')
)

In [169]:
def simResetZoom(new):
    simScat = simPlot.marks[3]
    simLin = simPlot.marks[2]
    refreshDimensions(simScat.x, simScat.y, simScat, simLin)
    simUpdateDistances(None)

In [170]:
simResetZoomButton.on_click(simResetZoom)

In [171]:
simCompressZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Zoom to selected area',
    icon='compress',
    layout = widgets.Layout(width='50px')
)

In [172]:
def simCompressZoom(new):
    if simScat.selected != None:
        if len(simScat.selected) > 2:
            dataX = np.take(simScat.x, simScat.selected)
            dataY = np.take(simScat.y, simScat.selected)   
            refreshDimensions(dataX, dataY, simScat, simLin)
            simUpdateDistances(None)
            sim_zoom_interacts.value = None

In [173]:
simCompressZoomButton.on_click(simCompressZoom)

In [174]:
pointsToolbar = widgets.HBox([], layout = widgets.Layout(align_self = 'center'))

addPointsButton = widgets.ToggleButton(
    description = 'Add points (OFF)',
    button_style = 'warning',
    style = style_1,
)

removePointsButton = widgets.ToggleButton(
    description = 'Remove points (OFF)',
    button_style = 'danger',
    style = style_1,
)

simSavePNGButton = widgets.ToggleButton(
    description = 'Save plot as png',
    button_style = '',
    style = style_1,
)

pointsToolbar.children = (addPointsButton, removePointsButton, )

In [175]:
def addPoint(change):

    if addPointsButton.value:
        removePointsButton.value = False
        addPointsButton.description = 'Add points (ON)'
        simPlot.interaction = None
        simScat.interactions = {'click' : 'add'}

    else:
        addPointsButton.description = 'Add points (OFF)'
        simScat.interactions = {'click' : None}
        simScat.observe(null)

In [176]:
addPointsButton.observe(addPoint, 'value')

In [177]:
def checkAddedPoint(change):
    if addPointsButton.value and not removePointsButton.value:
        simUpdateRegression(change)
        simUpdateDistances(change)

In [178]:
simScat.observe(checkAddedPoint, 'x')

In [179]:
def removePoint(change):

    if removePointsButton.value:
        addPointsButton.value = False
        removePointsButton.description = 'Remove points (ON)'
        simPlot.interaction = None
        simScat.on_element_click(deletePoint)
    else:
        removePointsButton.description = 'Remove points (OFF)'
        simScat.on_element_click(deletePoint)

In [180]:
def deletePoint(scat, point):

    if removePointsButton.value and len(simScat.x) > 2:
        simPlot.animation_duration=0
        i = point.get('data').get('index')
        dataX = np.delete(simScat.x, i)
        dataY = np.delete(simScat.y, i)
        [simScat.x, simScat.y] = dataX, dataY
        simUpdateRegression(None)
        simUpdateDistances(None)
        simPlot.animation_duration=500

In [181]:
removePointsButton.observe(removePoint, 'value')

In [182]:
def simSavePNG(change):
    if simSavePNGButton.value:
        simSavePNGButton.value = False
        simPlot.save_png('simulation.png')

In [183]:
simSavePNGButton.observe(simSavePNG, 'value')

In [184]:
editPointsBox = widgets.HBox([])

editPointsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)


editPointsAccordion.set_title(0, 'Edit points')
editPointsAccordion.children = (editPointsBox, )

In [185]:
dragXButton = widgets.ToggleButton(
    description = 'X axis (OFF)',
    button_style = '',
    style = style_1,
)

dragYButton = widgets.ToggleButton(
    description = 'Y axis (OFF)',
    button_style = '',
    style = style_1,
)

editPointsBox.children = (widgets.Label('Make points draggable:'), dragXButton,dragYButton,)

In [186]:
def drag(change):
    
    if dragXButton.value:
        dragXButton.description = 'X axis (ON)'
        simScat.restrict_x = True

    elif dragXButton.value == False:
        dragXButton.description = 'X axis (OFF)'
        simScat.restrict_x = False

    if dragYButton.value:
        dragYButton.description = 'Y axis (ON)'
        simScat.restrict_y = True

    elif dragYButton.value == False:
        dragYButton.description = 'Y axis (OFF)'
        simScat.restrict_y = False
        
    if dragXButton.value or dragYButton.value:
        simScat.enable_move = True     
        if dragXButton.value and dragYButton.value:
            simScat.restrict_x = False
            simScat.restrict_y = False
    else:
        simScat.enable_move = False     

In [187]:
dragXButton.observe(drag, 'value')
dragYButton.observe(drag, 'value')

In [188]:
simRegressionOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

simRegressionOptionsAccordion.set_title(0, 'Regression')

In [189]:
simRegressionOptionsBox = widgets.VBox([])

simRegressionBox = widgets.VBox([])

simShowRegressionCheckbox = widgets.Checkbox(
    value=False,
    description='Show regression',
    layout = widgets.Layout(width = '200px')
)

simRegressionColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#fad87c',
    disabled=False
)

simRegressionOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

simRegressionBox.children = (widgets.HTML(value='<h4>Edit regression:</h4>'), widgets.HBox([simRegressionColor, simRegressionOpacity]))

simRegressionOptionsBox.children = (simShowRegressionCheckbox,)
simRegressionOptionsAccordion.children = (simRegressionOptionsBox,)

In [190]:
def callUpdateRegression(a, b):
    simUpdateRegression(None)

In [191]:
def simShowRegression(change):
    if simShowRegressionCheckbox.value:
        simLin.visible = True
        simScat.on_drag_end(callUpdateRegression)
        simUpdateRegression(change)
        simUpdateDistances(change)

    else:
        simLin.visible = False

In [192]:
simShowRegressionCheckbox.observe(simShowRegression, 'value')

In [193]:
def simChangeRegressionColor(change):
    simLin.colors = [simRegressionColor.value]

In [194]:
simRegressionColor.observe(simChangeRegressionColor, 'value')

In [195]:
def simChangeRegressionOpacity(change):
    simLin.opacities = [simRegressionOpacity.value]

In [196]:
simRegressionOpacity.observe(simChangeRegressionOpacity, 'value')

In [197]:
simRegressionDecimals = widgets.Dropdown(
    options = [i for i in range(0,14)],
    value = 2,
    description='<h4>Number of decimals: </h4>',
    style = style_1
)

In [198]:
simRegressionOrder = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=5.0,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(width = '200px'),
)

simRegressionErrorText = widgets.HTML(
    value = "",
)

simRegressionValues = widgets.VBox()

In [199]:
def simUpdateRegression(change=None):

    if simRegressionExpression not in simRegressionOptionsBox.children:
    
        simRegressionOptionsBox.children = [widgets.HBox([simShowRegressionCheckbox, widgets.Label('Order of regression'), simRegressionOrder]),
                                            simRegressionExpression,
                                            simRegressionValues,
                                            simRegressionDecimals,
                                            widgets.HTML(value='<br>'),
                                            widgets.HBox([simShowDistancesCheckbox,simShowEstimatedCheckbox]),
                                            widgets.HTML(value='<hr>'),
                                            simRegressionBox,
                                            simDistancesBox,
                                            simEstimatedBox
                                           ]                                         

    if simScat.selected == None:
        
        dataX = simScat.x
        dataY = simScat.y
        
    else:

        if len(simScat.selected) < 3:

            dataX = simScat.x
            dataY = simScat.y

        else:

            dataX = np.take(simScat.x, simScat.selected)
            dataY = np.take(simScat.y, simScat.selected)    
       
    
    simLin.x = np.linspace(start=np.min(dataX), stop=np.max(dataX), num=1000)
    
    try:
        if len(dataX) > simRegressionOrder.value + 1:
            poly, cov = np.polyfit(dataX, dataY, simRegressionOrder.value, cov=True)
            simRegressionErrorText.value  = ''
        
        else:
            poly = np.polyfit(dataX, dataY, len(dataX) - 1)
            cov = np.zeros([len(poly),len(poly)])

            if simRegressionOrder.value >= len(simScat.x):
                simRegressionErrorText.value = "<span style='color:red'><b>You need more points for a regression of order " + str(simRegressionOrder.value) + "</b></span>"
            else:
                simRegressionErrorText.value = ''
    except:
        return None
    
    simLin.y = np.polyval(poly, simLin.x)
    
    if sim_squareSel.brushing == False:
    
        expression = '\(y = \;\)'
        maxPower = len(poly)-1

        decimals = simRegressionDecimals.value
        chars = 'abcdef'

        errors = np.sqrt(np.diag(cov))

        simRegressionValues.children = []

        for i in range(len(poly)):

            coef = np.around([ poly[i],],decimals)[0]

            error = np.around([errors[i],], decimals)[0]

            power = maxPower-i

            if power == 1:

                expression = expression + '\(' + chars[i] + '\; x \; + \; \)' 

            elif power == 0:
                expression = expression + '\(' + chars[i] + ' \)'

            else:
                expression = expression + '\(' + chars[i] + '\; x^{' + str(power) + '} \; + \; \)'

            s = '\(' + chars[i] + '\; = \; ' + str(coef) + '\; \pm \;' + str(error) + '\)'
            simRegressionValues.children = simRegressionValues.children + (widgets.HTMLMath(value = s , description = ''), )

        simRegressionExpression.value = expression

        errors = np.sqrt(np.diag(cov))
        errorsExp = ''

        for error in errors:
            error = np.around([error,], decimals)[0]
            errorsExp = errorsExp + '\(' + str(error) + '\; \; \; \)'

        simErrorsExpression.value = errorsExp    

        simUpdateDistances(change)

In [200]:
simRegressionOrder.observe(simUpdateRegression, 'value')

In [201]:
simRegressionDecimals.observe(simUpdateRegression, 'value')

In [202]:
simScat.observe(simUpdateRegression, 'selected')

In [203]:
sim_squareSel.observe(simUpdateRegression, 'brushing')

In [204]:
simRegressionExpression = widgets.HTMLMath(
    value = '\(y =\)',
    description = 'Expression:'
)

In [205]:
simErrorsExpression  = widgets.HTMLMath(
    value = '',
    description = 'Errors:'
)

In [206]:
def simUpdateDistances(change):

    scaleX = simScat.scales.get('x')
    scaleY = simScat.scales.get('y')
    
    simDist.scales = {'x': scaleX, 'y': scaleY}
    simScatEstimated.scales = {'x': scaleX, 'y': scaleY}
    
    x = simScat.x
    y = simScat.y
    
    try:
        yInterp = np.interp(x, simLin.x, simLin.y) 
    
    except:
        return None
    
    simScatEstimated.x = x
    simScatEstimated.y = yInterp    
    
    xVal = []
    yVal = []

    for i in range(len(x)):
        for j in range(2):
            xVal.append(x[i])
        yVal.append(min(y[i],yInterp[i]))
        yVal.append(max(y[i],yInterp[i]))
        
    xDef = []
    yDef = []

    for i in range(0, len(xVal), 2):
        xDef.append(np.array([xVal[i], xVal[i+1]]))
        yDef.append(np.array([yVal[i], yVal[i+1]]))
        
    simDist.x = xDef
    simDist.y = yDef

In [207]:
simDistancesBox = widgets.VBox([])

simShowDistancesCheckbox = widgets.Checkbox(
    value=False,
    description='Show distances',
)

simDistancesColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#cdb4c8',
    disabled=False
)

simDistancesOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='350px')
)

simDistancesBox.children = (widgets.HTML(value='<h4>Edit distances:</h4>'), widgets.HBox([simDistancesColor, simDistancesOpacity]))

In [208]:
def simShowDistances(change):
    
    if simShowDistancesCheckbox.value:
        simDist.visible = True
        simUpdateRegression(change)
        
    else:
        simDist.visible = False

In [209]:
simShowDistancesCheckbox.observe(simShowDistances, 'value')

In [210]:
def simChangeDistancesColor(change):
    simDist.colors = [simDistancesColor.value for elem in simDist.x]

In [211]:
simDistancesColor.observe(simChangeDistancesColor, 'value')

In [212]:
def simChangeDistancesOpacity(change):
    simDist.opacities = [simDistancesOpacity.value for elem in simDist.x]

In [213]:
simDistancesOpacity.observe(simChangeDistancesOpacity, 'value')

In [214]:
simEstimatedBox = widgets.VBox([])

simShowEstimatedCheckbox = widgets.Checkbox(
    value=False,
    description='Show estimated points',
)

simEstimatedColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#f87878',
    disabled=False
)

simEstimatedOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='350px')
)

simEstimatedBox.children = (widgets.HTML(value='<h4>Edit estimated points:</h4>'), widgets.HBox([simEstimatedColor, simEstimatedOpacity]))

In [215]:
def simShowEstimated(change):
    
    if simShowEstimatedCheckbox.value:
        simScatEstimated.visible = True
        simUpdateRegression(change)
        
    else:
        simScatEstimated.visible = False

In [216]:
simShowEstimatedCheckbox.observe(simShowEstimated, 'value')

In [217]:
def simChangeEstimatedColor(change):
    simScatEstimated.colors = [simEstimatedColor.value]

In [218]:
simEstimatedColor.observe(simChangeEstimatedColor, 'value')

In [219]:
def simChangeEstimatedOpacity(change):
    simScatEstimated.default_opacities = [simEstimatedOpacity.value]

In [220]:
simEstimatedOpacity.observe(simChangeEstimatedOpacity, 'value')

In [221]:
simText = widgets.HTML(
    value = "<br><h2>Your simulation:</h2>"    
)

In [222]:
saveSimButton = widgets.ToggleButton(
    description = 'Save simulation',
    button_style = '',
    style = style_1,
)

In [223]:
simName = widgets.Text(
    value='',
    placeholder="Simulation's name:",
    description="<h4>Name: </h4>",
    disabled=False,
)

In [224]:
def saveSim(change):
    if saveSimButton.value:
        saveSimButton.value = False
        pb.addPlot(simPlot, simName.value)
        simName.value = ''

In [225]:
saveSimButton.observe(saveSim, 'value')

In [226]:
discardButton = widgets.ToggleButton(
    description = 'Discard changes',
    button_style = '',
    style = style_1,
)

In [227]:
def discardChanges(change):
    if discardButton.value:
        discardButton.value = False
        changeSim(change)

In [228]:
discardButton.observe(discardChanges, 'value')

In [229]:
simPlotBox.children = (widgets.HBox([simPlot, widgets.VBox([widgets.HTML(value='<br/>'), sim_zoom_interacts, simCompressZoomButton, simResetZoomButton], layout = widgets.Layout(width='10%'))]), )

In [230]:
simBlock.children = [simText, widgets.HBox([simDropdown,deleteSimButton], layout = widgets.Layout(align_self = 'center')), simPlotBox, pointsToolbar, widgets.HBox([simName, saveSimButton,discardButton]), changeViewButton, editPointsAccordion, simRegressionOptionsAccordion, ]

In [231]:
savedBlock = widgets.VBox([], layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))

In [232]:
savedText = widgets.HTML(
    value = "<br><h2>Your plots:</h2>"    
)

In [233]:
plotDropdown = widgets.Dropdown(
    options = [],
    #value = 2,
    description='<h4>Choose your plot: </h4>',
    style = style_1
)

In [234]:
def refreshPlotDropdown():
    plotDropdown.options = pb.getPlots()

In [235]:
def changePlot(change):
    name = plotDropdown.value
    plot = pb.getPlot(name)
    savedBlock.children = [savedText, widgets.HBox([plotDropdown, deletePlotButton], layout = widgets.Layout(align_self = 'center')), plot]

In [236]:
plotDropdown.observe(changePlot, 'value')

In [237]:
deletePlotButton = widgets.ToggleButton(
    description = 'Delete plot',
    button_style = '',
    style = style_1,
)

In [238]:
def deletePlot(change):
    if deletePlotButton.value:
        deletePlotButton.value = False
        
        name = plotDropdown.value
        index = 0
        
        for i in range(len(plotDropdown.options)):
            opt = plotDropdown.options[i]
            if opt == name:
                index = i
                break
        
        index = index - 1
        
        plotDropdown.value = plotDropdown.options[index]
        pb.removePlot(name)
        
        refreshPlotDropdown()

In [239]:
deletePlotButton.observe(deletePlot, 'value')

In [240]:
savedBlock.children = [savedText, plotDropdown]

In [241]:
mainBlock